In [9]:
# Para llevar un registro de los cambios
%pip install -q comet_ml
import comet_ml
comet_ml.init(project_name='Tarea 03')

In [10]:
# Librerías
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Input
from tensorflow.keras.optimizers import RMSprop, SGD
from tensorflow.keras import regularizers
from keras.callbacks import ModelCheckpoint

In [11]:
experiment = comet_ml.Experiment(
    auto_histogram_weight_logging=True,
    auto_histogram_gradient_logging=True,
    auto_histogram_activation_logging=True,
    log_code=True,
)

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/gurumasterahb/tarea-03/741d1a3cba7644e3921ef116374a2b34
COMET INFO:   Uploads:
COMET INFO:     environment details : 1
COMET INFO:     filename            : 1
COMET INFO:     installed packages  : 1
COMET INFO:     notebook            : 2
COMET INFO:     os packages         : 1
COMET INFO:     source_code         : 1
COMET INFO: 
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Couldn't find a Git repository in '/content' nor in any parent directory. Set `COMET_GIT_DIRECTORY` if your Git Reposit

In [12]:
# Carga el dataset de MNIST
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [13]:
# Convierte y normaliza los datos de entrada
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [14]:
# to_categorical convierte las etiquetas (train, test) en matrices one-hot
num_classes = 10
y_trainc = keras.utils.to_categorical(y_train, num_classes)
y_testc = keras.utils.to_categorical(y_test, num_classes)

In [ ]:
  # Diccionario con los parámetros que se registrarán en comet
  parameters = {
      'epochs' : 30,
      'batch_size' : 10,
      'learning_rate' : 0.010,
      'optimizer' : 'RMSprop',
      'loss' : 'categorical_crossentropy'
  }
  experiment.log_parameters(parameters)

In [16]:
# Creación del modelo
model = Sequential()
# Es como una "primera capa" de tres capas
model.add(Input(shape = (28,28))) # Matrices de entrada de 28x28 (imáegenes)
model.add(Flatten()) # Convierte las matrices de 28x28 en matrices de 784x1
model.add(Dense(784, activation = 'sigmoid'))
model.add(Dense(30, activation = 'sigmoid')) # Segunda capa
model.add(Dense(10, activation = 'softmax')) # Tercera capa

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 784)               615440    
                                                                 
 dense_1 (Dense)             (None, 30)                23550     
                                                                 
 dense_2 (Dense)             (None, 10)                310       
                                                                 
Total params: 639300 (2.44 MB)
Trainable params: 639300 (2.44 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [17]:
# Checkpoint que guarda el mejor modelo (cuando la función de costo alcanza un mínimo)
checkpoint = ModelCheckpoint(filepath = 'best_model.hdf5', monitor = 'val_loss', verbose = 1, save_best_only = True, mode = 'min')
# Guarda el modelo cuando la precisión alcanza un máximo
# checkpoint = ModelCheckpoint(filepath = 'best_model.hdf5', monitor = 'val_acc', verbose = 1, save_best_only = True, mode = 'max')

In [18]:
model.compile(loss = parameters['loss'], optimizer = parameters['optimizer'], metrics = ['accuracy'])
model.fit(x_train, y_trainc, batch_size = parameters['batch_size'], epochs = parameters['epochs'], verbose = 1, validation_data = (x_test, y_testc), callbacks = [checkpoint])

COMET INFO: Ignoring automatic log_parameter('verbose') because 'keras:verbose' is in COMET_LOGGING_PARAMETERS_IGNORE


Epoch 1/30
6000/6000 [==============================] - ETA: 0s - loss: 0.3552 - accuracy: 0.9038
Epoch 1: val_loss improved from inf to 0.18077, saving model to best_model.hdf5
6000/6000 [==============================] - 71s 12ms/step - loss: 0.3552 - accuracy: 0.9038 - val_loss: 0.1808 - val_accuracy: 0.9460
Epoch 2/30
   1/6000 [..............................] - ETA: 1:15 - loss: 0.0443 - accuracy: 1.0000

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


6000/6000 [==============================] - ETA: 0s - loss: 0.1403 - accuracy: 0.9585
Epoch 2: val_loss improved from 0.18077 to 0.12603, saving model to best_model.hdf5
6000/6000 [==============================] - 58s 10ms/step - loss: 0.1403 - accuracy: 0.9585 - val_loss: 0.1260 - val_accuracy: 0.9614
Epoch 3/30
5998/6000 [============================>.] - ETA: 0s - loss: 0.0956 - accuracy: 0.9717
Epoch 3: val_loss improved from 0.12603 to 0.09505, saving model to best_model.hdf5
6000/6000 [==============================] - 62s 10ms/step - loss: 0.0956 - accuracy: 0.9717 - val_loss: 0.0951 - val_accuracy: 0.9707
Epoch 4/30
5996/6000 [============================>.] - ETA: 0s - loss: 0.0740 - accuracy: 0.9781
Epoch 4: val_loss improved from 0.09505 to 0.08842, saving model to best_model.hdf5
6000/6000 [==============================] - 59s 10ms/step - loss: 0.0739 - accuracy: 0.9781 - val_loss: 0.0884 - val_accuracy: 0.9749
Epoch 5/30
6000/6000 [==============================] - ETA:

In [19]:
experiment.log_model('MNIST1', 'best_model.hdf5')
experiment.end()

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/gurumasterahb/tarea-03/51b34f9074aa49ad8e20ca5e3a1525c1
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     accuracy [30]                  : (0.9038333296775818, 0.9999333620071411)
COMET INFO:     batch_accuracy [18000]         : (0.10000000149011612, 1.0)
COMET INFO:     batch_loss [18000]             : (4.1723228605405893e-07, 2.58369517326355)
COMET INFO:     epoch_duration [30]            : (57.15656397700013, 71.85111222299997)
COMET INFO:     loss [30]                      : (0.0002506444579921663, 0.35521626472473145)
COMET INFO:     val_accuracy [30]              : (0.9459999799728394, 0.9842000007629395)
COMET IN